In [ ]:
from feed_forward_network import FeedForwardNetwork
from lstm import LSTM
from transformer import Transformer

from preprocessing import *
from utils import *
from features import *

import pandas as pd

from sklearn.preprocessing import StandardScaler

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.set_per_process_memory_fraction(0.)
else:
    device = torch.device("cpu")
device

In [ ]:
raw_data = pd.read_csv('consumption_and_temperatures.csv')
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])

seq_len = 0
scale_output=True
target_column = 'NO1_consumption'

features_to_add_ffn = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {
            "shift_max": 10,
            "column_to_shift": "NO1_temperature",
            "new_column_name": "temp"
        }
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO1_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    ),
    (
        get_yesterday_target_mean,
        {'target': 'NO1_consumption'}
    )
]

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()

_, _, (X_test, y_test), (_, scalerOutputMethod1), _ = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add_ffn,
        seq_len=seq_len,
        scale_output=scale_output,
        scalerInputMethod=scaler1
    )

path1 = "FeedForwardNet-2024-03-19-loss-0.041835472"
path1 = "models/" + path1

model1 = FeedForwardNetwork(input_size=X_test.shape[1])
model1.load_state_dict(torch.load(path1))
model1.to(device)

y_test1, y_pred1 = predict(model1, scalerOutputMethod1, (X_test, y_test))

In [ ]:
seq_len = 72

features_to_add_lstm = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO1_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    )
]

_, _, (X_test, y_test), (_, scalerOutputMethod2), _ = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add_lstm,
        seq_len=seq_len,
        scale_output=scale_output,
        scalerInputMethod=scaler2
    )

path2 = "LSTM-no-prev-y-2024-03-19-loss-0.07680506"
path2 = "models/" + path2

input_size = X_test.shape[2]
hidden_size = 11
num_layers = 3

model2 = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, device=device)
model2.load_state_dict(torch.load(path2))
model2.to(device)

y_test2, y_pred2 = predict(model2, scalerOutputMethod2, (X_test, y_test))

In [ ]:
seq_len = 48
features_to_add_transformer = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO1_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    )
]

_, _, (X_test, y_test), (_, scalerOutputMethod3), _ = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add_transformer,
        seq_len=seq_len,
        scale_output=scale_output,
        scalerInputMethod=scaler3,
    )

path3 = "Transformer-2024-03-20-loss-0.035382744"
path3 = "models/" + path3

model3 = Transformer(input_size=X_test.shape[2])
model3.load_state_dict(torch.load(path3))
model3.to(device)

y_test3, y_pred3 = predict(model3, scalerOutputMethod3, (X_test, y_test))

In [ ]:
preds = {
    'ffn': torch.Tensor(y_pred1),
    'lstm': torch.Tensor(y_pred2),
    'transformer': torch.Tensor(y_pred3)
}

targets = {
    'ffn': torch.Tensor(y_test1),
    'lstm': torch.Tensor(y_test2),
    'transformer': torch.Tensor(y_test3)
}

summary_bar_plot(preds=preds, targets=targets)
